<a href="https://colab.research.google.com/github/averyrair/ChessBAKEN/blob/MoveSelect/MoveSelector/MoveSelectorBAKEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#@title Install Dependencies

!pip install chess
#!pip install stockfish
#!apt install stockfish

In [61]:
#@title Import Directories

from google.colab import drive
# For this to work, you need to have the "Chess Bot BAKEN" project shared with
# the SAME email you linked this Colab to, presumably your GitHub email address.
drive.mount('/content/drive', force_remount=1)
chess_dir = '/content/drive/Shareddrives/Chess Bot BAKEN'

Mounted at /content/drive


In [62]:
#@title Import Libraries

import os
import chess
import chess.pgn
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
#@title Define Custom Board Representation

def customBoardRep(board):

  return board.fen().split(" ")[0]

In [55]:
#@title Import PGN and Export CSV

import chess.pgn
#if not os.path.exists(os.path.join(chess_dir, 'moveSelectorDataset.csv')):
if 0:
  pgn_dir = os.path.join(chess_dir, 'KingBase2019-pgn')
  pgnFiles = [os.path.join(pgn_dir, f) for f in os.listdir(pgn_dir) if f.endswith('.pgn')]

  # Read every PGN file in the directory.
  iterator = 0
  for pgnFile in pgnFiles:
    print(pgnFile)
    pgn = open(pgnFile)

    # Prepare CSV to export dataset.
    with open(os.path.join(chess_dir, 'moveSelectorDataset' + pgnFile[74:81] +'.csv'), 'w') as csvfile:
      csvwriter = csv.writer(csvfile)
      #csvwriter.writeheader(['Label', 'Input'])

      # Read every game in the PGN file.
      while True:
        game = chess.pgn.read_game(pgn)
        if game is None: break
        iterator += 1
        if iterator % 500 == 0:
          print(iterator)
          #break
        board = game.board()

        # New data point for every move in the game.
        gameMoves = list(game.mainline_moves())
        for i in range(len(gameMoves)-1):
          board.push(gameMoves[i])
          legalMoves = ""
          for m in list(board.legal_moves):
            legalMoves = legalMoves + str(m) + " "
          legalMoves = legalMoves.removesuffix(" ")
          #print(gameMoves[i+1], legalMoves, customBoardRep(board))
          csvwriter.writerow([gameMoves[i+1], customBoardRep(board), legalMoves])

In [56]:
#@title Define Fully-Connected Neural Network

class myFCN(nn.Module):
  def __init__(self, inSize, hiddenSizes, outSize):
    super().__init__()
    self.inSize = inSize
    self.hiddenSize = hiddenSizes
    self.outSize = outSize

    self.lin1 = nn.Linear(inSize, hiddenSizes[0])
    self.lin2 = nn.Linear(hiddenSizes[0], hiddenSizes[1])
    self.lin3 = nn.Linear(hiddenSizes[1], outSize)

    self.bn1 = nn.BatchNorm1d(hiddenSizes[0])
    self.bn2 = nn.BatchNorm1d(hiddenSizes[1])

    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.bn1(self.lin1(x)))
    x = self.relu(self.bn2(self.lin2(x)))
    x = self.relu(self.lin3(x))
    return x

In [64]:
#@title Initialize Fully-Connected Neural Network

currentModelFile = 'MoveSelectorV1.0.pt'

moveSelector = myFCN(inSize=2560, hiddenSizes=[2200, 2200], outSize=1792)
if (os.path.exists(os.path.join(chess_dir, currentModelFile))):
  moveSelector = torch.load(os.path.join(chess_dir, currentModelFile), weights_only=False)
  moveSelector.eval()
  print("Loaded Model: " + currentModelFile)
print(moveSelector)

myFCN(
  (lin1): Linear(in_features=2560, out_features=2200, bias=True)
  (lin2): Linear(in_features=2200, out_features=2200, bias=True)
  (lin3): Linear(in_features=2200, out_features=1792, bias=True)
  (bn1): BatchNorm1d(2200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(2200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)


In [58]:
#@title Define Dataset

class ChessDataset(Dataset):
    def __init__(self, chessgame_file, chess_dir, transform=None, target_transform=None):
        self.game_labels = pd.read_csv(chessgame_file)
        self.chess_dir = chess_dir  # This is not used. Everything is in the chessgame file.
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.game_labels)

    def __getitem__(self, idx):
        label = self.game_labels.iloc[idx, 0]
        gameState = self.game_labels.iloc[idx, 1]
        moves = self.game_labels.iloc[idx, 2]

        return gameState, moves, label

In [59]:
#@title Define Encode/Decode Functions

# The number of possible pieces and square states (8*8*6*2)
NUM_BOARD_NODES = 768
# The number of possible moves from any square to any other
NUM_MOVE_NODES = 1792


# Determines whether any piece could ever move from one square to another
# Note: pass in start and end coordinates as tuples containing two ints
def canMove(start, end):
    # check duplicate
    if start[0] == end[0] and start[1] == end[1]:
        return False

    # check row or col
    if start[0] == end[0] or start[1] == end[1]:
        return True

    # check diagonal
    if abs(start[0]-end[0]) == abs(start[1]-end[1]):
        return True

    # check knight
    knightCases = [(1, 2), (1, -2), (-1, 2), (-1, -2),
                   (2, 1), (2, -1), (-2, 1), (-2, -1)]
    for c in knightCases:
        if start[0] + c[0] == end[0] and start[1] + c[1] == end[1]:
            return True

    return False


# Generate the mapping matrix (only needs to be done once)
def getMappingMatrix():
    moveMap = [[[[0 for l in range(8)] for k in range(
        8)] for j in range(8)] for i in range(8)]
    offset = 0
    for i in range(8):
        for j in range(8):
            for k in range(8):
                for l in range(8):
                    if canMove((i, j), (k, l)):
                        moveMap[i][j][k][l] = offset
                        offset += 1
    return moveMap

# Encode a board position and legal moves into a binary array
def encodeSelector(boardFEN, moves, moveMap):
    # Encode board position
    encodedBoard = [0 for _ in range(NUM_BOARD_NODES)]
    rowsFEN = boardFEN.split('/')
    for i in range(8):
        col = 0
        for j in range(len(rowsFEN[i])):
            if rowsFEN[i][j].isdigit():
                col += ord(rowsFEN[i][j])-48
            else:
                match rowsFEN[i][j]:
                    case 'P':
                        idx = (8*6*2*col) + (6*2*i) + (0*2) + 0
                        encodedBoard[idx] = 1
                    case 'N':
                        idx = (8*6*2*col) + (6*2*i) + (1*2) + 0
                        encodedBoard[idx] = 1
                    case 'B':
                        idx = (8*6*2*col) + (6*2*i) + (2*2) + 0
                        encodedBoard[idx] = 1
                    case 'R':
                        idx = (8*6*2*col) + (6*2*i) + (3*2) + 0
                        encodedBoard[idx] = 1
                    case 'Q':
                        idx = (8*6*2*col) + (6*2*i) + (4*2) + 0
                        encodedBoard[idx] = 1
                    case 'K':
                        idx = (8*6*2*col) + (6*2*i) + (5*2) + 0
                        encodedBoard[idx] = 1
                    case 'p':
                        idx = (8*6*2*col) + (6*2*i) + (0*2) + 1
                        encodedBoard[idx] = 1
                    case 'n':
                        idx = (8*6*2*col) + (6*2*i) + (1*2) + 1
                        encodedBoard[idx] = 1
                    case 'b':
                        idx = (8*6*2*col) + (6*2*i) + (2*2) + 1
                        encodedBoard[idx] = 1
                    case 'r':
                        idx = (8*6*2*col) + (6*2*i) + (3*2) + 1
                        encodedBoard[idx] = 1
                    case 'q':
                        idx = (8*6*2*col) + (6*2*i) + (4*2) + 1
                        encodedBoard[idx] = 1
                    case 'k':
                        idx = (8*6*2*col) + (6*2*i) + (5*2) + 1
                        encodedBoard[idx] = 1
                col += 1

    # Encode possible moves
    encodedMoves = [0 for _ in range(NUM_MOVE_NODES)]
    for m in moves.split(' '):
        startCol = ord(str(m)[0])-97  # e.g. c -> 2
        startRow = ord(str(m)[1])-49  # e.g. 8 -> 7
        endCol = ord(str(m)[2])-97
        endRow = ord(str(m)[3])-49
        encodedMoves[(moveMap[startCol][startRow][endCol][endRow])] = 1

    # Combine into single list
    return encodedBoard + encodedMoves


# Decode board position from a binary array of length 768
def decodePosition(input):
    # Create matrix of board
    boardMatrix = [['.' for _ in range(8)] for _ in range(8)]

    # Populate board
    for i in range(len(input)):
        if input[i] == 1:
            piece = i % 12
            # Piece: P = 0, R = 1, N = 2, B = 3, Q = 4, K = 5
            # Color: W = 0, B = 1
            pieceLabel = '.'
            match piece:
                case 0:
                    pieceLabel = 'P'
                case 1:
                    pieceLabel = 'p'
                case 2:
                    pieceLabel = 'N'
                case 3:
                    pieceLabel = 'n'
                case 4:
                    pieceLabel = 'B'
                case 5:
                    pieceLabel = 'b'
                case 6:
                    pieceLabel = 'R'
                case 7:
                    pieceLabel = 'r'
                case 8:
                    pieceLabel = 'Q'
                case 9:
                    pieceLabel = 'q'
                case 10:
                    pieceLabel = 'K'
                case 11:
                    pieceLabel = 'k'
            boardMatrix[int(i / (6*2)) % 8][int(i / (8*6*2))] = pieceLabel

    # Convert to FEN notation
    boardFEN = ''
    for i in range(len(boardMatrix)):
        empty = 0
        for j in range(len(boardMatrix[i])):
            if boardMatrix[i][j] == '.':
                empty += 1
            else:
                if empty > 0:
                    boardFEN += str(empty)
                    empty = 0
                boardFEN += boardMatrix[i][j]
        if empty > 0:
            boardFEN += str(empty)
        if i < len(boardMatrix) - 1:
            boardFEN += '/'

    return chess.Board(boardFEN)


# Decode moves from a binary array of length 1792
# Returns a list of moves with start and end squares
def decodeMoves(input, moveMap):
    moves = []
    for h in range(len(input)):
        if input[h] > 0: # NOTE: may need to change this condition based on selector NN output
            # Search moveMap to find the actual move squares
            found = False
            for i in range(8):
                for j in range(8):
                    for k in range(8):
                        for l in range(8):
                            if moveMap[i][j][k][l] == h:
                                moves.append(chr(i+97) + chr(j+49) + chr(k+97) + chr(l+49))
                                found = True
                                break
                        if found:
                            break
                    if found:
                        break
                if found:
                    break
    return moves

def decodeLegalMoves(original, input, moveMap, threshold = 0):
    indices = [i for i, x in enumerate(original) if x == 1]
    moves = []
    for h in indices:#range(len(input)):
        if input[h] > threshold: # NOTE: may need to change this condition based on selector NN output
            # Search moveMap to find the actual move squares
            found = False
            for i in range(8):
                for j in range(8):
                    for k in range(8):
                        for l in range(8):
                            if moveMap[i][j][k][l] == h:
                                moves.append((chr(i+97) + chr(j+49) + chr(k+97) + chr(l+49), input[h].item()))
                                found = True
                                break
                        if found:
                            break
                    if found:
                        break
                if found:
                    break
    return moves

def moveLabel(moveMade, moveMap):
    encodedMoves = [0 for _ in range(NUM_MOVE_NODES)]
    startCol = ord(str(moveMade)[0])-97  # e.g. c -> 2
    startRow = ord(str(moveMade)[1])-49  # e.g. 8 -> 7
    endCol = ord(str(moveMade)[2])-97
    endRow = ord(str(moveMade)[3])-49
    encodedMoves[(moveMap[startCol][startRow][endCol][endRow])] = 1
    return encodedMoves

def getMoveLoss(modelOutput, expectedOutput, weighting = NUM_MOVE_NODES):
    totalLoss = 0
    for i in range(NUM_MOVE_NODES):
        if expectedOutput[i] == 1:
            # model should output value close to 1
            totalLoss += abs(1 - modelOutput[i]) * weighting
            pass
        else:
            # model should output value close to 0
            totalLoss += abs(modelOutput[i])
            pass
    return totalLoss

### TESTING ###

# Test the encoder and decoder
def testEncodeDecode():
    # Encode board
    moveMap = getMappingMatrix()
    with open(os.path.join(chess_dir, 'Move Selector Dataset', 'moveSelectorDatasetD70-D99.csv'), 'r') as file:
        data = list(csv.reader(file))
    curr = data[0]
    encoded = encodeSelector(curr[1], curr[2], moveMap)

    # Decode board
    decodedBoard = decodePosition(encoded[0:NUM_BOARD_NODES])
    decodedMoves = decodeMoves(encoded[-NUM_MOVE_NODES:], moveMap)

    # Compare original and decoded position/moves
    print('Original position and legal moves: ')
    print(chess.Board(curr[1]))
    print(sorted(curr[2].split(' ')))
    print('---------------')
    print('Decoded position and legal moves: ')
    print(decodedBoard)
    print(decodedMoves)

#testEncodeDecode()

In [65]:
#@title Train Model

dataset = ChessDataset(chessgame_file=os.path.join(chess_dir, 'Move Selector Dataset/moveSelectorDatasetD70-D99.csv'), chess_dir=chess_dir)
moveMap = getMappingMatrix()

optim = torch.optim.SGD(moveSelector.parameters(), lr=0.0001, momentum=0.9)

epochs = 4
for e in range(epochs):
  runningLoss = 0
  batchSize = 32

  # Number of batches
  for b in range(int(len(dataset)/batchSize)):

    X = []
    labels = []

    # Make the batch
    for i in range(batchSize):
      offset = b * batchSize
      X.append(encodeSelector(dataset[i + offset][0], dataset[i + offset][1], moveMap))
      labels.append(dataset[i + offset][2])

    optim.zero_grad()

    X = torch.tensor(X).float()
    output = moveSelector(X)

    loss = 0
    for i in range(batchSize):
      expectedOutput = moveLabel(labels[i], moveMap)
      loss += getMoveLoss(output[i], expectedOutput, NUM_MOVE_NODES*10)

    loss.backward()
    runningLoss += loss.item()
    optim.step()

  else:
    print(e, runningLoss/len(dataset))
    #print(X[0])
    #print(output[0])

    #print(decodeLegalMoves(X[0][-NUM_MOVE_NODES:], output[0], moveMap))


0 39104765.82843819
1 17972.184507238308
2 17912.836163697106
3 17903.29830178174


In [48]:
#@title Save Model

torch.save(moveSelector, os.path.join(chess_dir, currentModelFile))

In [69]:
X = []
labels = []
for i in range(batchSize):
  X.append(encodeSelector(dataset[i][0], dataset[i][1], moveMap))
  labels.append(dataset[i][2])

X = torch.tensor(X).float()
testData = moveSelector(X)

torch.set_printoptions(threshold=torch.inf)
print(testData[6])
print(labels[6])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 